# Lesson 8 - APL functions, part 3: ⍋⍒⍳

We're in the middle of our functions' marathon. Next up is `⍋`, called [grade up](http://help.dyalog.com/latest/index.htm#Language/Primitive%20Functions/Grade%20Up%20Monadic.htm). Monadic `⍋` takes a simple (non-nested) array and returns the indices of the major cells reordered so that they would order the array.

Easiest to understand with an example: 

In [7]:
⎕IO←1

In [1]:
⍋3 1 4 1 5

2 4 1 3 5

This means that the second element (1) is the smallest, then the fourth (1), then the first (3), etc. So, we can use this to sort the array: 

In [2]:
3 1 4 1 5[⍋3 1 4 1 5]

1 1 3 4 5

It works on high-rank arrays too: 

In [3]:
3 2⍴2 7 1 8 2 8
⍋3 2⍴2 7 1 8 2 8 

2 7
1 8
2 8

2 1 3

So the first is row 2 `(1 8)` then row 1 `(2 7)` then row 3 `(2 8)`. It works on characters too, where it grades in Unicode point order: 

In [4]:
5 2⍴'HelloWorld'
⍋5 2⍴'HelloWorld'
(5 2⍴'HelloWorld')[⍋5 2⍴'HelloWorld';] 

He
ll
oW
or
ld

1 5 2 3 4

He
ld
ll
oW
or

In [12]:
4 2 2⍴'Hello World PPCG'   
⍋4 2 2⍴'Hello World PPCG'  ⍝ layer grade up

He
ll

o 
Wo

rl
d 

PP
CG

1 4 2 3

Layer 1, layer 4, layer 2, layer 3:

In [13]:
{⍵[⍋⍵;;]}4 2 2⍴'Hello World PPCG'

He
ll

PP
CG

o 
Wo

rl
d

`⍋⍋` is the cardinal numbers:

In [14]:
⍋'PPCG' 
⍋⍋'PPCG' 

3 4 1 2

3 4 1 2

So P is the third, P is the fourth, C is the first, and G is the second. Applying `⍋` to a permutation inverts it (swaps between cardinal and grade). Another way to think about it is that `⍋` is the indices of cells in the order that would sort them. `⍋⍋` is the position each will take when sorted. If you think about it hard, you'll see why ⍋ swaps back and forth between these two. 

Here's an example where the grade and the cardinals differ:

In [17]:
⍋'random'
⍋⍋'random' 
⍋⍋⍋'random'  ⍝ grading the cardinals takes us back to grade

2 4 6 3 5 1

6 1 4 2 5 3

2 4 6 3 5 1

`⍋` once is what order the elements would be in when sorted and `⍋` twice is the indices that each element would go to.

Dyadic `⍋` is for character arrays only, and it grades as if the left argument was the alphabet: 

In [18]:
{⍵['aeioubcdfghjklmnpqrstvwxyz'⍋⍵]}'helloworld'

eoodhlllrw

If characters are missing from the alphabet, they will be considered after the alphabet, and equivalent:

In [19]:
'abcdefgh'⍋'hawl'

2 1 3 4

Dyadic `⍋` can also use multiple levels of sorting: 

In [23]:
⍉↑'aeiou' 'bcdfghjklmnpqrstvwxyz'

ab
ec
id
of
ug
 h
 j
 k
 l
 m
 n
 p
 q
 r
 s
 t
 v
 w
 x
 y
 z

This 2D "alphabet" means that all vowels should come before all consonants, and only if otherwise the same, the vertical order will be considered. 

In [21]:
{⍵[(⍉↑'aeiou' 'bcdfghjklmnpqrstvwxyz')⍋⍵]}'helloworld'

eoodhlllrw

This sorted all vowels before all consonants, and only then did it sort the vowels and the consonants. You can have up to 15 levels of sorting using this. If a letter occurs more than once, then its first occurrence rules. This is useful to fill gaps in (e.g.) columns of unequal height.

There is also `⍒`, which is [grade down](http://help.dyalog.com/latest/index.htm#Language/Primitive%20Functions/Grade%20Down%20Monadic.htm), which follows the pattern of `⍋`, but sorts the other way.

Monadic `⍳` is the [index generator](http://help.dyalog.com/latest/index.htm#Language/Primitive%20Functions/Index%20Generator.htm). `⍳a` generates an array of shape a where the elements are the indices for that element: 

In [24]:
⍳10
⍳2 4

1 2 3 4 5 6 7 8 9 10

┌───┬───┬───┬───┐
│1 1│1 2│1 3│1 4│
├───┼───┼───┼───┤
│2 1│2 2│2 3│2 4│
└───┴───┴───┴───┘

Any bets on what `⍳0` gives?

In [26]:
]display ⍳0

┌⊖┐
│0│
└~┘

The empty numeric list. What about `⍳0 0`?

In [27]:
]display ⍳0 0

┌⊖──────┐
⌽ ┌→──┐ │
│ │0 0│ │
│ └~──┘ │
└∊──────┘

This is the same as `0 0⍴⍬`; a 0x0 empty numeric matrix.

The dyadic version `A⍳B` is [index-of](http://help.dyalog.com/latest/index.htm#Language/Primitive%20Functions/Index%20Of.htm). It finds the first occurrence of the major cells of `B` in the major cells of `A`: 

In [28]:
'hello'⍳'l'
'hello'⍳'lo' 

3

3 5

If a cell is not a member, it will return a number one higher than the number of elements:

In [29]:
'hello'⍳'x'

6

In [30]:
(3 2⍴'abcdef')⍳(2 2⍴'cdxy')

2 4

So the "cd" row is the second one, and the "xy" row is not there. This behaviour for elements that are not there is really useful for supplying a "default":

In [32]:
'First' 'Second' 'Third' 'Missing'['abc'⍳'cdab']

┌─────┬───────┬─────┬──────┐
│Third│Missing│First│Second│
└─────┴───────┴─────┴──────┘